# Imports

In [ ]:
import asyncio
import traceback
import warnings
from collections import defaultdict, namedtuple
from collections.abc import Mapping, Sequence
from functools import partial
from glob import glob
from importlib import reload
from numbers import Number
from operator import getitem

import cachetools
import dask
import distributed
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import param
import parambokeh
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq
import qgrid
import scipy
import skimage.morphology
import streamz
import streamz.dataframe as sdf
import zarr
from bokeh.models.tools import HoverTool, TapTool
from cytoolz import *
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from holoviews.streams import Selection1D, Stream, param
from tqdm import tnrange, tqdm, tqdm_notebook
from traitlets import All

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common
import diagnostics
import geometry
import image
import metadata
import trench_detection
import trench_detection.core
import trench_detection.hough
import trench_segmentation.watershed
import ui
import util
import workflow

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Restore data

In [ ]:
%store -r trench_points
%store -r trench_diag
%store -r trench_bboxes
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # job_extra=['--exclude=compute-e-16-181,compute-e-16-186'],
    # interface='ib0',
    memory="3GB",
    local_directory="/tmp",
    cores=4,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
# cluster = LocalCluster(n_workers=1)
# client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
cluster.stop_jobs(cluster.running_jobs.keys())

In [ ]:
cluster.scheduler.stop_services()
cluster.scheduler.stop()

In [ ]:
del cluster

In [ ]:
# client = Client()

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2',
#                 '/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2', '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']
nd2_filenames = glob("/n/scratch2/jqs1/fidelity/all/180405_*.nd2") + glob(
    "/n/scratch2/jqs1/fidelity/all/TrErr*.nd2"
)

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)
image_limits = workflow.get_filename_image_limits(metadata)

# Reload

In [ ]:
def _check():
    import os

    return os.path.exists(
        "/n/scratch2/jqs1/fidelity/all/180405_txnerr_loweronly_fast.nd2"
    )


[k for k, v in client.run(_check).items() if v is False]

In [ ]:
def do_reload():
    from importlib import reload

    import diagnostics
    import image
    import trench_detection
    import util
    import workflow

    # reload(util)
    # reload(trench_detection.hough)
    # reload(diagnostics)
    reload(workflow)
    # reload(image)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :, ["MCHERRY"], 0], :]

In [ ]:
len(frames_to_process)

## Run trench finding

In [ ]:
%%time
find_trenches_diag = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=True, pandas=True
)
trench_info_futures = {
    idx: client.submit(
        find_trenches_diag, client.submit(workflow.get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
progress(trench_info_futures)

In [ ]:
client.cancel(trench_info_futures)

In [ ]:
def as_completed(obj, with_results=True):
    if isinstance(obj, Mapping):
        futures = obj.values()
        dask_to_keys = {future.key: k for k, future in obj.items()}
    else:
        raise NotImplementedError
    for res in distributed.as_completed(futures, with_results=with_results):
        if with_results:
            future, result = res
            yield dask_to_keys[future.key], future, result
        else:
            future = res
            yield dask_to_keys[future.key], future

In [ ]:
trench_info = {}
for key, fut, res in as_completed(trench_info_futures):
    trench_info[key] = res
    client.cancel(fut)

In [ ]:
%%time
trench_info = util.apply_map_futures(
    client.gather, trench_info_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
%%time
trench_points, trench_diag, trench_err = workflow.unzip_trench_info(trench_info)

In [ ]:
len(trench_points)

In [ ]:
%%time
%store trench_points
%store trench_diag

## Analysis

In [ ]:
bad_angle = trench_diag["find_trench_lines.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_pitch = (trench_diag["find_trench_lines.hough_2.peak_func.pitch"] - 24).abs() > 1
bad_pitch.sum()

In [ ]:
selected = trench_diag[bad_pitch]  # trench_diag[bad_angle | bad_period]

In [ ]:
frame_stream.event(_df=selected.index.to_frame(index=False))

In [ ]:
%%time
trench_points_good = trench_points[~util.multi_join(trench_points.index, bad_pitch)]

In [ ]:
(len(trench_points_good), len(trench_points_good) / len(trench_points))

In [ ]:
%%time
trench_bbox_futures = []
for _, trenches in trench_points_good.groupby(["filename", "position", "t"]):
    trench_bbox_futures.append(
        client.submit(workflow.get_trench_bboxes, trenches, image_limits)
    )

In [ ]:
%%time
trench_bbox_results = util.apply_map_futures(
    client.gather, trench_bbox_futures, predicate=lambda x: x.status == "finished"
)
trench_bboxes = pd.concat(
    [trench_points_good, pd.concat(trench_bbox_results, axis=0)], axis=1
)

In [ ]:
%%time
%store trench_bboxes

In [ ]:
%store -r trench_bboxes

In [ ]:
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]
# trench_bboxes_t0.index = trench_points_good_t0.index.droplevel('t')

# Trench finding QA

In [ ]:
selected = all_frames

In [ ]:
FrameStream = ui.DataFrameStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

In [ ]:
frame = workflow.get_nd2_frame_anyargs(**dict(frame_stream.get_param_values()))

In [ ]:
tp, diag, _ = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=False
)(frame)

In [ ]:
ui.show_plot_browser(diag)

# Segmentation

In [ ]:
selected_trenches_segmentation = trench_bboxes_t0[
    trench_bboxes_t0[("info", "hough_value")] > 90
].loc[IDX[:, :, ["MCHERRY"], 0, :, :], :]

In [ ]:
(len(trench_bboxes_t0), len(selected_trenches_segmentation) / len(trench_bboxes_t0))

In [ ]:
# frames_to_analyze = all_frames.loc[IDX[:,:1,['MCHERRY','YFP'],1:5],:]
frames_to_analyze = all_frames.loc[IDX[:, :1, ["MCHERRY", "YFP"], 1:5], :]

In [ ]:
(
    len(frames_to_analyze),
    len(all_frames.loc[IDX[:, :, ["MCHERRY", "YFP"], :], :]) / len(frames_to_analyze),
)

In [ ]:
%%time
labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.3", "p0.5", "p0.7", "p0.9", "p0.95"): partial(
        np.percentile, q=(30, 50, 70, 90, 95)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

analyze_trench_func = partial(
    workflow.analyze_trenches,
    framewise_funcs=framewise_funcs,
    trenchwise_funcs=trenchwise_funcs,
    labelwise_funcs=labelwise_funcs,
    regionprops=False,
    segment_func=trench_segmentation.watershed.segment_trench,
)
analyze_trench_func = compose(
    list, partial(map, pa.RecordBatch.from_pandas), analyze_trench_func
)

analyze_trench_func = partial(client.submit, analyze_trench_func)

analysis_futures = workflow.analyze_frames_and_trenches(
    selected_trenches_segmentation, frames_to_analyze, analyze_trench_func
)

# display(trenchwise_df)

In [ ]:
client.cancel(analysis_futures)

In [ ]:
progress(analysis_futures)

In [ ]:
%lprun -f pd.core.groupby.groupby.BaseGrouper.get_iterator -f workflow.analyze_frames_and_trenches -f util.iter_index workflow.analyze_frames_and_trenches(selected_trenches_segmentation, frames_to_analyze, analyze_trench_func)

In [ ]:
%prun workflow.analyze_frames_and_trenches(selected_trenches_segmentation, frames_to_analyze, analyze_trench_func)

In [ ]:
client.cancel(analysis_futures)

## New new collect

In [ ]:
asyncio.get_event_loop()

In [ ]:
asyncio.get_event_loop().set_debug(True)

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [ ]:
async def gather_stream(source, futures):
    ac = distributed.as_completed(futures, with_results=False)
    async for future in ac:
        await source.emit(future)


from distributed.client import default_client
from tornado import gen


@streamz.Stream.register_api()
class gather_and_cancel(streamz.Stream):
    def __init__(self, upstream, stream_name=None, client=None, cancel=True):
        if client is None:
            client = default_client()
        self.client = client
        self.cancel = cancel
        streamz.Stream.__init__(self, upstream, stream_name=stream_name)

    @gen.coroutine
    def update(self, x, who=None):
        result = yield self.client.gather(x, asynchronous=True)
        # if self.cancel:
        #    self.client.cancel(x)
        result2 = yield self._emit(result)
        raise gen.Return(result2)


def sink_to_arrow(batches, sinks, writers):
    for i, batch in enumerate(batches):
        if i not in writers:
            sinks[i] = pa.BufferOutputStream()
            writers[i] = pa.RecordBatchStreamWriter(sinks[i], batch.schema)
        writers[i].write_batch(batch)


source = streamz.Stream(asynchronous=True)

stream_sinks = {}
stream_writers = {}

# source.rate_limit(0.0004).timed_window(1).map(partial(gather_and_cancel, cancel=True)).flatten().sink(partial(sink_to_arrow, sinks=stream_sinks, writers=stream_writers))
# source.rate_limit(0.0004).timed_window(1).gather_and_cancel(client=client, cancel=False).flatten().sink(partial(sink_to_arrow, sinks=stream_sinks, writers=stream_writers))
source.gather_and_cancel(client=client, cancel=True).sink(
    lambda x: print(np.random.randint(10))
)

t = asyncio.get_event_loop().create_task(gather_stream(source, analysis_futures))

In [ ]:
isinstance(AssertionError(), Exception)

In [ ]:
from distributed.client import default_client
from tornado import gen


@streamz.Stream.register_api()
class gather_and_cancel(streamz.Stream):
    def __init__(self, upstream, stream_name=None, client=None, cancel=True):
        if client is None:
            client = default_client()
        self.client = client
        self.cancel = cancel
        streamz.Stream.__init__(self, upstream, stream_name=stream_name)

    @gen.coroutine
    def update(self, x, who=None):
        try:
            result = yield self.client.gather(x, asynchronous=True)
        except Exception as exc:
            result = exc
        if self.cancel:
            self.client.cancel(x)
        result2 = yield self._emit(result)
        raise gen.Return(result2)


source = streamz.Stream(asynchronous=True)

stream_sinks = {}
stream_writers = {}

source2 = source.rate_limit(0.001).timed_window(0.1)
source3 = streamz.buffer(source2, 100).gather_and_cancel(client=client, cancel=True)
source3.filter(lambda x: not isinstance(x, Exception)).sink(
    partial(sink_to_arrow, sinks=stream_sinks, writers=stream_writers)
)
exceptions = source3.filter(lambda x: isinstance(x, Exception)).sink_to_list()
t = asyncio.get_event_loop().create_task(source.emit(analysis_futures[0]))

In [ ]:
t.cancel()

In [ ]:
t.

In [ ]:
%%time
pa.open_stream(stream_sinks[2].getvalue()).read_pandas()

## New collect

In [ ]:
stream_sinks = {}
stream_writers = {}

In [ ]:
analysis_res = []

In [ ]:
async def gather_stream(source, futures):
    ac = distributed.as_completed(futures, with_results=False)
    async for future in ac:
        await source.emit(future)
        # source.emit(future)


def gather_and_cancel(futures, cancel=True):
    results = client.gather(futures)
    if cancel:
        client.cancel(futures)
    return results


# async def gather_and_cancel_async(futures, cancel=True):
#     results = await client.gather(futures, asynchronous=True)
#     if cancel:
#         asyncio.ensure_future(client.cancel(futures, asynchronous=True))
#     return results

# async def gather_cancel_map(futures, func, cancel=True):
#     results = await gather_and_cancel_async(futures, cancel=cancel)
#     func(results)


def pandas_to_arrow(dfs, sinks, writers):
    for i, df in enumerate(dfs):
        batch = pa.RecordBatch.from_pandas(df)
        if i not in writers:
            sinks[i] = pa.BufferOutputStream()
            writers[i] = pa.RecordBatchStreamWriter(sinks[i], batch.schema)
        writers[i].write_batch(batch)


source = streamz.Stream(asynchronous=True)

# source.rate_limit(0.1).timed_window(1).map(partial(gather_and_cancel, cancel=True)).sink(analysis_res.extend)
# source.rate_limit(0.0004).timed_window(1).sink(partial(gather_cancel_map, func=analysis_res.extend))
# source.rate_limit(0.0004).timed_window(1).map(partial(gather_and_cancel, cancel=True)).flatten().sink(partial(pandas_to_arrow, sinks=stream_sinks, writers=stream_writers))
# source.rate_limit(0.1).timed_window(1).sink(lambda x: 0)
source.rate_limit(0.0004).timed_window(1).map(
    partial(gather_and_cancel, cancel=True)
).flatten().sink(partial(pandas_to_arrow, sinks=stream_sinks, writers=stream_writers))

t = asyncio.get_event_loop().create_task(gather_stream(source, analysis_futures))

In [ ]:
t

In [ ]:
t.cancel()

In [ ]:
len(analysis_res)

In [ ]:
t

In [ ]:
stream_writers

In [ ]:
# started 8pm sharp

In [ ]:
%%time
for i, sink in stream_sinks.items():
    out = pa.OSFile(
        "/n/scratch2/jqs1/fidelity/all/output/analysis100_stream_{}.arrow".format(i),
        "w",
    )
    out.write(sink.getvalue())
    out.close()

In [ ]:
%%time
pa.open_stream(stream_sinks[2].getvalue()).read_pandas()

In [ ]:
pa.MemoryMappedFile(, 'w')

In [ ]:
t.cancel()

In [ ]:
t

In [ ]:
len(analysis_res)

In [ ]:
util.apply_map_futures(
    client.gather, analysis_futures, predicate=lambda x: x.status == "error"
)

In [ ]:
len(labelwise_df)

In [ ]:
labelwise_df.info(memory_usage="deep")

# Analysis

## Load data

In [ ]:
%%time
framewise_df = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis50_full_0.parquet"
).to_pandas()

In [ ]:
%%time
trenchwise_df = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis50_full_1.parquet"
).to_pandas()

In [ ]:
%%time
labelwise_df = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis50_full_2.parquet"
).to_pandas()

In [ ]:
labelwise_df.info(memory_usage="deep")

In [ ]:
# labelwise_df.index.names = ['filename', 'position', 't', 'trench_set', 'trench', 'label']
# labelwise_df.sort_index(inplace=True)

In [ ]:
len(labelwise_df)

In [ ]:
framewise_df.head()

In [ ]:
trenchwise_df.head()

In [ ]:
labelwise_df.head()

## Basic stats

In [ ]:
# labelwise_df2 = labelwise_df.copy()
labelwise_df.columns = ["_".join(col).strip() for col in labelwise_df.columns.values]

In [ ]:
labelwise_df.loc[:, "MCHERRY_regionprops_area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells = labelwise_df2[
    (50 < labelwise_df2["MCHERRY_regionprops_area"])
    & (labelwise_df2["MCHERRY_regionprops_area"] < 300)
]

In [ ]:
selected_cells.loc[:, "MCHERRY_regionprops_area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "MCHERRY_labelwise_p0.5"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "YFP_labelwise_p0.5"].plot(kind="hist", bins=100, log=True)

In [ ]:
labelwise_df[:10000].hvplot(
    x="MCHERRY_labelwise_p0.5", y="YFP_labelwise_p0.5", kind="scatter"
)

In [ ]:
yfp_cols = [c for c in labelwise_df.columns if c.startswith("YFP_labelwise_p")]
mcherry_cols = [c for c in labelwise_df.columns if c.startswith("MCHERRY_labelwise_p")]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], yfp_cols
]

In [ ]:
d.hvplot.scatter("t", "YFP_labelwise_p0.95", by="trench")

In [ ]:
%%opts Scatter (size=3)
hv.Layout(
    [
        d.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
d2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, :, :, :], yfp_cols
]

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
hv.Layout(
    [
        d2.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
# labelwise_df.index.get_level_values('position')

In [ ]:
# class NYCTaxi(hv.streams.Stream):
#     alpha = param.Magnitude(default=0.75, doc="Opacity")
#     colormap = param.ObjectSelector(default="fire", objects=
#         hpu.list_cmaps(category='Uniform Sequential', bg='dark'))
#     location = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])

#     def make_view(self, x_range, y_range, **kwargs):
#         pts   = hv.Points(taxi, [self.location+'_x', self.location+'_y'])
#         trips = datashade(pts, cmap=hpu.process_cmap(self.colormap),
#             x_range=x_range, y_range=y_range, dynamic=False, **dopts)
#         return tiles.options(alpha=self.alpha) * trips
# explorer = NYCTaxi(name="NYC Taxi Trips")
# parambokeh.Widgets(explorer, callback=explorer.event)
# hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
def get_plot(pos):
    c = "YFP_labelwise_p0.9"
    d = labelwise_df.loc[
        IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", pos, :, :, :], c
    ]
    return d.hvplot.scatter("t", c, by="trench", width=500, height=300)


hv.HoloMap(
    {
        pos: get_plot(pos)
        for pos in labelwise_df.index._get_level_values(1, unique=True)[:20]
    }
)

In [ ]:
labelwise_df.head().loc[:, cols]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], cols
]
m = pd.melt(d.loc[:, cols])
# m.index = d.index
# m

# Plot tapping

In [ ]:
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
label_stream = LabelStream()
box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
p = d.hvplot.scatter("t", "YFP_labelwise_p0.95").options(tools=["tap", "hover"])
ui.selection_to_stream(p, label_stream)
p

In [ ]:
def cb(**kwargs):
    dat = labelwise_df.loc[
        workflow.stream_slice(
            labelwise_df.index.names, kwargs, t=slice(None), label=slice(None)
        ),
        :,
    ]
    plot = dat.hvplot.scatter("t", "YFP_labelwise_p0.95").options(
        tools=["tap", "hover"]
    )
    # ui.selection_to_stream(plot, label_stream)
    return plot


ui.viewer(cb, label_stream)

## Transcription errors

In [ ]:
selected_cells.head()

In [ ]:
selected_cells.index[0][:-1]

In [ ]:
trench_bboxes.head()

In [ ]:
selected_cells.head()

In [ ]:
selected_cells.index[0][2:-1]

In [ ]:
selected_trenches = trench_bboxes.xs(
    (*selected_cells.index[0][:2], "MCHERRY", 0, *selected_cells.index[0][3:-1]),
    drop_level=False,
)

In [ ]:
selected_trenches.head()

In [ ]:
workflow.get_trench_stacks(selected_trenches, all_frames, image_limits)

# Trench UI

In [ ]:
trenchwise_df.head(10)

In [ ]:
trench_bboxes.head(20)

In [ ]:
trench_bboxes[trench_bboxes[("info", "hough_value")] > 90].loc[
    IDX[:, :, ["MCHERRY"], 0, :, :], :
].head(10)

In [ ]:
trench_bboxes.head(10)

In [ ]:
selected_trenches_segmentation.head(10)

In [ ]:
# LabelStream = ui.MultiIndexStream.define('LabelStream', trench_bboxes.index)
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
# LabelStream = ui.MultiIndexStream.define('LabelStream', selected_trenches_segmentation.index)
label_stream = LabelStream()

box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
label_stream.contents

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=500 height=300]
# image_callback = partial(ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached)
# p = ui.trench_set_viewer(trench_bboxes_t0, label_stream, channel='MCHERRY', image_callback=image_callback).options({'Bounds': dict(tools=['tap'])})
p = ui.get_trench_set_overlay(
    trench_bboxes_t0,
    channel="MCHERRY",
    **{
        k: label_stream.contents[k]
        for k in ("filename", "position", "t", "trench_set", "trench")
    },
)
p = p.options({"Bounds": dict(tools=["tap"])})
s = Selection1D(source=p)
p

In [ ]:
s

In [ ]:
s

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=800 height=150]
image_callback = partial(
    ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached
)
(
    ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="MCHERRY", image_callback=image_callback
    )
    + ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="YFP", image_callback=image_callback
    )
).cols(1)

In [ ]:
%%opts Layout [normalize=False]
%%output size=100
hover = HoverTool(
    tooltips=[
        ("(x,y)", "(@x{0[.]0}, @y{0[.]0})"),
        ("value", "@z"),
    ]
)
cb = compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)
# cb = workflow.get_trench_image
(
    ui.trench_viewer(trench_bboxes, label_stream, channel="MCHERRY", image_callback=cb)
    + ui.trench_viewer(trench_bboxes, label_stream, channel="YFP", image_callback=cb)
).cols(1)

In [ ]:
import bokeh

In [ ]:
compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)(
    trench_bboxes,
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    3,
    "MCHERRY",
    0,
    1,
    56,
)

In [ ]:
yfp = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 3, 1, 61],
    ("YFP", "labelwise", "mean"),
]

In [ ]:
yfp.plot.hist(bins=50)

In [ ]:
yfp2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3],
    ("YFP", "labelwise", "p0.95"),
]

In [ ]:
yfp2[yfp2 < 300].plot.hist(bins=50, log=True)

In [ ]:
yfp2[yfp2 > 200]

In [ ]:
yfp2.loc[IDX[:, 1, 60]]

In [ ]:
yfp2.swaplevel(0, 1).swaplevel(1, 2).sort_index()

In [ ]:
yfp2[yfp2 > 130].groupby(["trench_set", "trench"]).filter(lambda x: x.count() > 10)

In [ ]:
yfp[yfp > 130]

In [ ]:
labelwise_df.xs(
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 2, 1, 56],
    drop_level=False,
)["YFP"]

In [ ]:
labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 0, 2, 1, 56], "MCHERRY"
]

In [ ]:
plt.imshow(
    workflow.get_trench_image(
        trench_bboxes,
        channel="MCHERRY",
        **dissoc(label_stream.contents, "_df", "label"),
    )
)

In [ ]:
workflow.get_nd2_frame(
    channel="MCHERRY",
    **util.get_keys(label_stream.contents, "filename", "position", "t"),
)

In [ ]:
f = partial(util.get_keys, keys=["label"])
f(label_stream.contents)

In [ ]:
label_stream.contents.keys()

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

# Memory

In [ ]:
from pympler import tracker

memory_tracker = tracker.SummaryTracker()
memory_tracker.print_diff()

In [ ]:
from pympler.classtracker import ClassTracker

tracker = ClassTracker()
tracker.track_object()

In [ ]:
import objgraph

In [ ]:
s = objgraph.by_type("IOStream")

In [ ]:
s[0]

In [ ]:
objgraph.show_backrefs(s[50])

In [ ]:
objgraph.show_most_common_types(50)